In [1]:
%matplotlib

import sys
sys.path.append('..')

from brian2 import defaultclock, ms, pA, nA, Hz, seed, mV, second
import numpy as np
from results import Results
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import seaborn as sns
import pandas as pd

from stats import moving_average, outlying
from scipy.stats import ttest_rel

Using matplotlib backend: Qt5Agg


INFO       Cache size for target "cython": 4937 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\mikejseay\.cython\brian_extensions" directory. [brian2]


In [11]:
# targetFile = 'buonoEphys_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-13-14-55_results'  # very nice but not the right params exactly
# targetFile = 'buonoEphys2_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal2_test_2021-08-16-14-26_results'  # very nice
# targetFile = 'buonoEphys2_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal2_test_2021-08-16-14-35_results'  # very nice
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-16-16-19_results'  # very nice (ben idea worked)
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-16-16-46_results'  # another example
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-17-11-09_results'  # good

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-17-14-13_results'  # no Exc2
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono2Weights2e3p025LogNormal_test_2021-08-17-14-16_results'  # yes Exc2

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono4Weights2e3p025LogNormal3__2021-08-23-14-49_results'  # yes Exc2
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_guessBuono4Weights2e3p025LogNormal3__2021-08-23-14-12_results'  # no Exc2

# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono4Weights2e3p05LogNormal3__2021-08-23-15-57_results'  # yes Exc2
# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono4Weights2e3p05LogNormal3__2021-08-23-15-52_results'  # no Exc2

# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono4Weights2e3p05LogNormal3__2021-08-23-16-05_results'  # yes Exc2, really good

# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono6Weights2e3p05Beta10__2021-08-24-11-16_results'  # no Exc2
# targetFile = 'buonoEphysBen1_2000_0p5_upPoisson_guessBuono6Weights2e3p05Beta10__2021-08-24-11-20_results'  # yes Exc2

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-04-17-21_results'  # best so far
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-05-21-49_results'  # control equivalent

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-08-14-28_results'  # good control, 30 s with 2 ups
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-08-14-43_results'  # good experimental, 30 s with 2 ups

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-09-10-47_results'  # modify conn 1: 10% removed
targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-09-10-10-15_results'  # modify conn 2: 10% removed, some added back

# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior__2021-10-04-13-54_results'  # 20% removed, added back
# targetFile = 'buonoEphysBen1_2000_0p25_upPoisson_resumePrior_EImod_2021-10-04-14-03_results'  # 20% removed, added back

targetFolder = 'C:/Users/mikejseay/Documents/BrianResults/'

In [21]:
plt.close('all')

In [12]:
R = Results()
R.init_from_file(targetFile, targetFolder)

print(R.p['simName'])

classicJercogEphysBuonoBen1


In [13]:
plt.close('all')

In [14]:
R.p['indsRecordStateExc']

[0, 1599, 100, 940, 1052, 1434]

In [15]:
R.p['indsRecordStateInh']

[0, 27]

In [16]:
R.calculate_PSTH()
R.calculate_voltage_histogram(removeMode=True, removeReset=True, useAllRecordedUnits=False, useExcUnits=1)
R.calculate_upstates()
if len(R.ups) > 0:
    R.reshape_upstates()
    R.calculate_FR_in_upstates()
    infoStr = 'average FR in upstate for Exc: {:.2f}, Inh: {:.2f}, average upDur: {:.2f}, upFreq: {:.2f}'.format(R.upstateFRExcHist.mean(), R.upstateFRInhHist.mean(), R.upDurs.mean(), R.ups.size / R.p['duration'] / Hz)
    print(infoStr)


R.calculate_voltage_histogram(useExcUnits=(2, 4,), useInhUnits=(0,))
R.reshape_upstates()

average FR in upstate for Exc: 3.87, Inh: 9.14, average upDur: 0.77, upFreq: 0.13


In [9]:
# special plot: two voltages for a 20 sec period with 3 up states (hopefully)
# e vs i

timePeriod = (0, 8)

f, ax = plt.subplots(2, 1, num=2, figsize=(16, 5), sharex=True, sharey=True)
R.plot_voltage_detail(ax[0], unitType='Exc', useStateInd=3)
R.plot_voltage_detail(ax[1], unitType='Inh', useStateInd=1)
R.plot_updur_lines(ax[1])
ax[1].set(xlim=timePeriod, xlabel='Time (s)')

f.savefig('benEphys1Voltage.pdf', transparent=True)

In [33]:
# special plot: two voltages for a 20 sec period with 3 up states (hopefully)
# e- vs e+

timePeriod = (49, 58)

f, ax = plt.subplots(2, 1, num=2, figsize=(9, 5), sharex=True, sharey=True)
R.plot_voltage_detail(ax[0], unitType='Exc', useStateInd=3)
R.plot_voltage_detail(ax[1], unitType='Exc', useStateInd=2, overrideColor='royalblue')
R.plot_updur_lines(ax[1])
ax[1].set(xlim=timePeriod, xlabel='Time (s)')

f.savefig('benEphys3Voltage.pdf', transparent=True)

In [10]:
# special plot: spike raster for a 20 sec period with 3 up states (hopefully)

timePeriod = (0, 8)

fig1, ax1 = plt.subplots(2, 1, num=3,
                         gridspec_kw={'height_ratios': [2, 1]},
                         figsize=(16, 5), sharex=True)
R.plot_spike_raster(ax1[0])
R.plot_firing_rate(ax1[1])
ax1[1].set_ylim(0, 40)
ax1[1].set(xlim=timePeriod, xlabel='Time (s)')

fig1.savefig('benEphys1Raster.pdf', transparent=True)

In [17]:
fig1, ax1 = plt.subplots(6, 1, num=1, figsize=(16, 9),
                         gridspec_kw={'height_ratios': [3, 1, 1, 1, 1, 1]},
                         sharex=True)
R.plot_spike_raster(ax1[0])  # uses RNG but with a separate random seed
R.plot_firing_rate(ax1[1])
ax1[1].set_ylim(0, 30)
R.plot_voltage_detail(ax1[2], unitType='Exc', useStateInd=0)
R.plot_updur_lines(ax1[2])
R.plot_voltage_detail(ax1[3], unitType='Inh', useStateInd=0)
R.plot_updur_lines(ax1[3])
R.plot_voltage_detail(ax1[4], unitType='Exc', useStateInd=1)
R.plot_updur_lines(ax1[4])
if 'useSecondPopExc' in R.p and R.p['useSecondPopExc']:
    R.plot_voltage_detail(ax1[5], unitType='Exc', useStateInd=2)
    R.plot_updur_lines(ax1[5])
ax1[5].set(xlabel='Time (s)')
R.plot_voltage_histogram_sideways(ax1[2], 'Exc')
R.plot_voltage_histogram_sideways(ax1[3], 'Inh')
fig1.suptitle(infoStr)

Text(0.5, 0.98, 'average FR in upstate for Exc: 3.87, Inh: 9.14, average upDur: 0.77, upFreq: 0.13')

In [46]:
fig1, ax1 = plt.subplots(5, 1, num=1, figsize=(16, 9),
                         gridspec_kw={'height_ratios': [3, 1, 1, 1, 1]},
                         sharex=True)
R.plot_spike_raster(ax1[0])  # uses RNG but with a separate random seed
R.plot_firing_rate(ax1[1])
ax1[1].set_ylim(0, 30)
R.plot_voltage_detail(ax1[2], unitType='Exc', useStateInd=5)
R.plot_voltage_detail(ax1[3], unitType='Inh', useStateInd=0)
R.plot_voltage_detail(ax1[4], unitType='Exc', useStateInd=2, overrideColor='royalblue')
R.plot_updur_lines(ax1[2])
R.plot_updur_lines(ax1[3])
R.plot_updur_lines(ax1[4])
ax1[4].set(xlabel='Time (s)')
R.plot_voltage_histogram_sideways(ax1[2], 'Exc')
R.plot_voltage_histogram_sideways(ax1[3], 'Inh')
R.plot_voltage_histogram_sideways(ax1[4], 'Exc')
fig1.suptitle(R.p['simName'])

Text(0.5, 0.98, 'classicJercogEphysBuonoBen1')

In [43]:
R.calculate_voltage_histogram(removeMode=True, useAllRecordedUnits=False, useExcUnits=2)

In [15]:
f, ax = plt.subplots(figsize=(6, 4))
# R.plot_voltage_histogram(ax)
# ax.plot(R.voltageHistCenters, (R.voltageHistExc + R.voltageHistInh)/2)
ax.plot(R.voltageHistCenters, R.voltageHistExc)
ax.set(xlabel='Voltage (mv)', ylabel='# of occurences')

[Text(0, 0.5, '# of occurences'), Text(0.5, 0, 'Voltage (mv)')]

In [18]:
R.calculate_upFR_units()

In [19]:
frInp = R.upstateFRExcUnits[:, :R.p['nUnitsToSpike']].mean(0)  # input pop
frEx2 = R.upstateFRExcUnits[:, R.p['nUnitsToSpike']:(R.p['nUnitsToSpike'] + R.p['nUnitsSecondPopExc'])].mean(0)  # secondary pop
frEx1 = R.upstateFRExcUnits[:, (R.p['nUnitsToSpike'] + R.p['nUnitsSecondPopExc']):].mean(0)  # normal pop
frInh = R.upstateFRInhUnits.mean(0)

frInpHat = frInp.mean()
frEx2Hat = frEx2.mean()
frEx1Hat = frEx1.mean()
frInhHat = frInh.mean()

frInpSTD = frInp.std()
frEx2STD = frEx1.std()
frEx1STD = frEx1.std()
frInhSTD = frInh.std()

print(frInpHat, frEx2Hat, frEx1Hat, frInhHat, )
print(frInpSTD, frEx2STD, frEx1STD, frInhSTD, )

frInpDF = pd.DataFrame(frInp, columns=('FR',))
frEx2DF = pd.DataFrame(frEx2, columns=('FR',))
frEx1DF = pd.DataFrame(frEx1, columns=('FR',))
frInhDF = pd.DataFrame(frInh, columns=('FR',))

frInpDF['unitType'] = 'Inp'
frEx2DF['unitType'] = 'Ex2'
frEx1DF['unitType'] = 'Ex1'
frInhDF['unitType'] = 'Inh'

frDF = pd.concat((frInpDF, frEx2DF, frEx1DF, frInhDF, ))

# frDF = pd.concat((frEx2DF, frEx1DF, frInhDF, ))

4.201520772828711 0.3603609544758373 5.366307916111041 12.931081976292495
0.6167987428108929 0.7335771194929483 0.7335771194929483 2.157713573405929


In [23]:
frEx1C = R.upstateFRExcUnits[:, (R.p['nUnitsToSpike'] + R.p['nUnitsSecondPopExc']):].mean(0)  # normal pop
frEx1CHat = frEx1C.mean()
frEx1CSTD = frEx1C.std()
frEx1CDF = pd.DataFrame(frEx1C, columns=('FR',))
frEx1CDF['unitType'] = 'Ex1C'

In [24]:
frDF = pd.concat((frInpDF, frEx2DF, frEx1DF, frInhDF, frEx1CDF))

# frDF = pd.concat((frEx2DF, frEx1DF, frInhDF, ))

In [51]:
frDF

,FR,unitType
0,4.000377,Inp
1,4.089369,Inp
2,4.588998,Inp
3,3.576164,Inp
4,3.288823,Inp
...,...,...
1395,3.988800,Ex1C
1396,4.458446,Ex1C
1397,4.445625,Ex1C
1398,4.431041,Ex1C


In [78]:
# Create an array with the colors you want to use
# colors = ["#006400", "#90EE90"]
# colors = ["#006400", "#00FFFF"]
colors = ["#00FFFF", "#4E69B2"]
# colors = ["#006400", "#90EE90", "#00FFFF"]
#00FFFF

# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))

In [28]:
f, ax = plt.subplots(figsize=(3.25, 4))
# sns.violinplot(x='unitType', y='FR', data=frDF, width=.6, palette=customPalette, order=('Ex1C', 'Ex1', 'Ex2'))
sns.violinplot(x='unitType', y='FR', data=frDF, width=.6, palette=customPalette, order=('Ex1', 'Ex2'))
# sns.stripplot(x="unitType", y="FR", data=frDF, size=4, color=".3", linewidth=0)
f.savefig('benEphys3Violin.pdf', transparent=True)

In [60]:
sns.swarmplot(x='unitType', y='FR', size=1.5, color='k', data=frDF, order=('Ex1', 'Ex2'))

In [64]:
f, ax = plt.subplots()
sns.boxplot(x='unitType', y='FR', data=frDF, width=.6, order=('Ex1', 'Ex2'), palette=customPalette)
# sns.stripplot(x="unitType", y="FR", data=frDF, size=4, color=".3", linewidth=0)

In [63]:
sns.stripplot(x='unitType', y='FR', size=1.5, color='k', data=frDF, order=('Ex1', 'Ex2'))

In [172]:
# 1-2 vs 2-3
# paradoxControlFrDF = pd.read_csv('C:/Users/mikejseay/PycharmProjects/spiking-upstates/paradox_control_results1.csv').set_index('trialIndex')
# paradoxExperimentFrDF = pd.read_csv('C:/Users/mikejseay/PycharmProjects/spiking-upstates/paradox_experiment_results1.csv').set_index('trialIndex')

# 2-3 vs 3-4
# paradoxControlFrDF = pd.read_csv('C:/Users/mikejseay/PycharmProjects/spiking-upstates/control_results2.csv').set_index('trialIndex')
# paradoxExperimentFrDF = pd.read_csv('C:/Users/mikejseay/PycharmProjects/spiking-upstates/experiment_results2.csv').set_index('trialIndex')

# 3-4 vs 4-5
# paradoxControlFrDF = pd.read_csv('C:/Users/mikejseay/PycharmProjects/spiking-upstates/control_4s_results.csv').set_index('trialIndex')
# paradoxExperimentFrDF = pd.read_csv('C:/Users/mikejseay/PycharmProjects/spiking-upstates/experimental_4s_results.csv').set_index('trialIndex')
# paradoxControlFrDF = paradoxControlFrDF.replace('3-4 s', '4-5 s').replace('2-3 s', '3-4 s')
# paradoxExperimentFrDF = paradoxExperimentFrDF.replace('3-4 s', '4-5 s').replace('2-3 s', '3-4 s')

# comparison = 2-3, experiment = 3-4
FrDF = pd.read_csv('C:/Users/mikejseay/PycharmProjects/spiking-upstates/multiMeta1_results.csv').set_index('trialIndex')

In [173]:
paradoxControlFrDF = FrDF[FrDF['currentAmplitude'] == '0. A']
paradoxExperimentFrDF = FrDF[FrDF['currentAmplitude'] == '6. pA']

In [174]:
# paradoxControlFrDF['Current Amplitude (nA)'] = '0 pA'
# paradoxExperimentFrDF['Current Amplitude (nA)'] = '8 pA'

In [175]:
outlyingIndicesControl = outlying(paradoxControlFrDF['FR'][:30], sd_thresh=2.5)
outlyingIndicesExperiment = outlying(paradoxExperimentFrDF['FR'][:30], sd_thresh=2.5)

outlyingIndices = set(np.concatenate((np.where(outlyingIndicesControl)[0], np.where(outlyingIndicesExperiment)[0])))

print(outlyingIndices)

paradoxControlFrDF2 = paradoxControlFrDF.drop(outlyingIndices)
paradoxExperimentFrDF2 = paradoxExperimentFrDF.drop(outlyingIndices)

paradoxDF = pd.concat((paradoxControlFrDF2, paradoxExperimentFrDF2))

paradoxDFEx = paradoxDF[paradoxDF['unitType'] == 'Ex']
paradoxDFInh = paradoxDF[paradoxDF['unitType'] == 'Inh']

{3, 5, 7}


In [176]:
print(outlyingIndices)

{3, 5, 7}


In [180]:
paradoxControlFrDF2 = paradoxControlFrDF.drop(outlyingIndices)
paradoxExperimentFrDF2 = paradoxExperimentFrDF.drop(outlyingIndices)

paradoxDF = pd.concat((paradoxControlFrDF2, paradoxExperimentFrDF2))

paradoxDFEx = paradoxDF[paradoxDF['unitType'] == 'Ex']
paradoxDFInh = paradoxDF[paradoxDF['unitType'] == 'Inh']

In [220]:
# colors = ["#00FFFF", "#4E69B2"]
# customPalette = sns.set_palette(sns.color_palette(colors))

useDF = paradoxDFInh
# level1 = 'currentAmplitude'
# level2 = 'timePeriod'
# level2order = ('2-3 s', '3-4 s')
level1 = 'timePeriod'
level2 = 'currentAmplitude'
level2order = useDF[level2].unique()
dvName = 'FR'
xValues = ([-0.25, 0.25], [0.75, 1.25])

# Draw a nested barplot by species and sex
g = sns.catplot(
#     data=paradoxDFEx,
    data=useDF,
    kind="box",
    x=level1, y=dvName, hue=level2,
    hue_order=level2order,
    # col='unitType', palette=customPalette,
)
g.despine(left=True)
g.set_axis_labels('Current Amplitude (pA)', "Firing Rate (Hz)")
g.fig.subplots_adjust(top=0.85)
g.fig.suptitle('Firing Rate during Up state prior to (2-3 s)\n and during (3-4 s) current injection')
g.legend.set_title(level2)
yMin, yMax = g.ax.get_ylim()
g.set(ylim=(np.floor(yMax / 2), yMax))

# try to superimpose the lines
# reshape the data

for level1Ind, level1Value in enumerate(useDF[level1].unique()):
    statsDF = useDF[useDF[level1] == level1Value]
    statsDF1 = statsDF[statsDF[level2] == level2order[0]]
    statsDF2 = statsDF[statsDF[level2] == level2order[1]]
    t, p = ttest_rel(statsDF1[dvName].values, statsDF2[dvName].values)
    statsText = 't = {:.2f}, -log10(p) = {:.2f}'.format(t, -np.log10(p))
    g.ax.text(level1Ind - 0.4, yMin + 1 - 2 * level1Ind, statsText)
    seriesLst = []
    for level2Value in level2order:
        boolSelection = (useDF[level1] == level1Value) & (useDF[level2] == level2Value)
        seriesLst.append(useDF[dvName][boolSelection].values.reshape(-1, 1))
    pairedArray = np.concatenate(seriesLst, 1)
    g.ax.plot(np.array(xValues[level1Ind]).reshape(-1, 1), pairedArray.T, color='k', lw=0.4, alpha=0.4)

In [135]:
colors = ["#00FFFF", "#4E69B2"]
customPalette = sns.set_palette(sns.color_palette(colors))

# Draw a nested barplot by species and sex
g = sns.catplot(
    data=paradoxDFInh, kind="violin",
    x="timePeriod", y="FR", hue="Current Amplitude (nA)", # col='unitType',
    palette=customPalette
)
g.despine(left=True)
g.set_axis_labels("Time Period", "Firing Rate (Hz)")
g.fig.subplots_adjust(top=0.85)
g.fig.suptitle('Firing Rate during Up state when Inh units receive current\ninjection (1-2 s) and afterward in its absence (2-3 s)')
g.legend.set_title("")

In [207]:
plt.close('all')

### PSTH analysis of paradoxical effect

In [221]:
import matplotlib
matplotlib.rcParams.update({'font.size': 14})

In [222]:
class Results(object): pass

In [280]:
# loadPath = 'C:/Users/mikejseay/PycharmProjects/spiking-upstates/results/multiMeta1L_PSTH.npz'  # up state regime
# loadPath = 'C:/Users/mikejseay/PycharmProjects/spiking-upstates/results/multiMetaHigh1_PSTH.npz'  # up state regime - high currents
# loadPath = 'C:/Users/mikejseay/PycharmProjects/spiking-upstates/results/multiMetaDestexheAI3_PSTH.npz'  # AI regime - Destexhe
# loadPath = 'C:/Users/mikejseay/PycharmProjects/spiking-upstates/results/multiMetaAI4_PSTH.npz'  # AI regime - Jercog
loadPath = 'C:/Users/mikejseay/PycharmProjects/spiking-upstates/results/multiMetaUpWeightsPoissonInput1_PSTH.npz'  # AI protocol, up state weights

npzObject = np.load(loadPath, allow_pickle=True)
R = Results()
for savedObjectName in npzObject.files:
    setattr(R, savedObjectName, npzObject[savedObjectName])

print(R.currentValues * 1e12)

[  0.  20.  40.  60.  80. 100. 120. 140. 160. 180. 200.]


In [281]:
timeVector = np.arange(0, 5.1, 0.01)[:-1]
histCenters = timeVector + (timeVector[1] - timeVector[0]) / 2

# check for outliers in the FR
# cut off is 2 Hz for Exc and 6 Hz for Inh

REMOVE_OUTLIERS = False
PLOT_FAILURES = False

paraStartTime = 3
paraEndTime = 4
paraStartInd = np.argmin(np.abs(histCenters - paraStartTime))
paraEndInd = np.argmin(np.abs(histCenters - paraEndTime))

paraPeriodTrialMeansExc = R.PSTHExc[:, :, paraStartInd:paraEndInd].mean(2)
paraPeriodTrialMeansInh = R.PSTHInh[:, :, paraStartInd:paraEndInd].mean(2)

paraPeriodTrialMeansExcNoFailures = paraPeriodTrialMeansExc.copy()
paraPeriodTrialMeansInhNoFailures = paraPeriodTrialMeansInh.copy()

if REMOVE_OUTLIERS:
    paraPeriodTrialMeansExcNoFailures[paraPeriodTrialMeansExc < 2] = np.nan
    paraPeriodTrialMeansInhNoFailures[paraPeriodTrialMeansInh < 6] = np.nan

# paraPeriodMeansExc = R.PSTHExc[:, :, paraStartInd:paraEndInd].mean((1, 2))
# paraPeriodMeansInh = R.PSTHInh[:, :, paraStartInd:paraEndInd].mean((1, 2))

paraPeriodMeansExc = np.nanmean(paraPeriodTrialMeansExcNoFailures, 1)
paraPeriodMeansInh = np.nanmean(paraPeriodTrialMeansInhNoFailures, 1)
paraPeriodSTDsExc = np.nanstd(paraPeriodTrialMeansExcNoFailures, 1)
paraPeriodSTDsInh = np.nanstd(paraPeriodTrialMeansInhNoFailures, 1)

In [282]:
# FR in the stim period as a function of current with outliers removed

f, ax = plt.subplots(1, 2, sharey=True, figsize=(10.5, 4.5))
ax[0].plot(R.currentValues * 1e12, paraPeriodMeansExc, color='cyan')
ax[1].plot(R.currentValues * 1e12, paraPeriodMeansInh, color='red')
ax[0].fill_between(R.currentValues * 1e12, paraPeriodMeansExc - paraPeriodSTDsExc, paraPeriodMeansExc + paraPeriodSTDsExc,
               alpha=0.5, color='cyan')
ax[1].fill_between(R.currentValues * 1e12, paraPeriodMeansInh - paraPeriodSTDsInh, paraPeriodMeansInh + paraPeriodSTDsInh,
               alpha=0.5, color='red')
ax[0].set(xlabel='Current Applied (pA)', ylabel='FR mean during current')
ax[1].set(xlabel='Current Applied (pA)')
f.subplots_adjust(left=0.1, bottom=0.15, right=0.95, top=0.95)

# number of failures per thing
outlierPSTHExc = paraPeriodTrialMeansExc < 2
outlierPSTHInh = paraPeriodTrialMeansInh < 6

if PLOT_FAILURES:
    f, ax = plt.subplots()
    ax.plot(R.currentValues * 1e12, outlierPSTHExc.sum(1) / 30, color='cyan')
    ax.plot(R.currentValues * 1e12, outlierPSTHInh.sum(1) / 30, color='red')
    ax.set(xlabel='Current Applied (pA)', ylabel='Proportion of up states ended')

PSTHExcOutNan = R.PSTHExc.copy()
PSTHExcOutNan[outlierPSTHExc, :] = np.nan

PSTHInhOutNan = R.PSTHInh.copy()
PSTHInhOutNan[outlierPSTHInh, :] = np.nan

In [283]:
(R.PSTHExc == 0).sum() / R.PSTHExc.size

0.12320652497469786

In [285]:
# average PSTH, no outliers removed

f, ax = plt.subplots(1, 2, sharex=True, figsize=(10.5, 4.5))

spaceCurrentValues = 1
movAvgWidth = 3
movAvgAdj = int(movAvgWidth / 2)

nCurrentValues = len(R.currentValues)
redColors = plt.cm.Reds(np.linspace(0, 1, nCurrentValues + 3))
blueColors = plt.cm.Blues(np.linspace(0, 1, nCurrentValues + 3))

for currentValueInd in range(0, nCurrentValues, spaceCurrentValues):
    currentValue = R.currentValues[currentValueInd]
    useLabel = str(int(currentValue * 1e12)) + ' pA'
    xVals = timeVector
    yValsExc = R.PSTHExc.mean(1)[currentValueInd, :]
    yValsInh = R.PSTHInh.mean(1)[currentValueInd, :]
#     xVals = timeVector[movAvgAdj:-movAvgAdj]
#     yValsExc = moving_average(R.PSTHExc.mean(1)[currentValueInd, :], movAvgWidth)
#     yValsInh = moving_average(R.PSTHInh.mean(1)[currentValueInd, :], movAvgWidth)
    ax[0].plot(xVals, yValsExc, label=useLabel, color=blueColors[currentValueInd + 3])
    ax[1].plot(xVals, yValsInh, label=useLabel, color=redColors[currentValueInd + 3])

ax[0].legend(loc='lower left')
ax[1].legend(loc='lower left')

# ax[0].set(xlabel='Time (s)', ylabel='E FR (Hz)', ylim=(0, 9))
# ax[1].set(xlabel='Time (s)', ylabel='I FR (Hz)', ylim=(0, 24))

ax[0].set(xlabel='Time (s)', ylabel='E FR (Hz)', ylim=(0, 15))
ax[1].set(xlabel='Time (s)', ylabel='I FR (Hz)', ylim=(0, 40))
f.subplots_adjust(left=0.1, bottom=0.15, right=0.95, top=0.95)
# ax[0].vlines(3, *ax[0].get_ylim(), color='k', ls='--', lw=1)
# ax[1].vlines(3, *ax[1].get_ylim(), color='k', ls='--', lw=1)

In [277]:
# average PSTH, outliers removed

f, ax = plt.subplots(2, 1, sharex=True, figsize=(7, 9))

movAvgWidth = 3
movAvgAdj = int(movAvgWidth / 2)

for currentValueInd, currentValue in enumerate(R.currentValues):
    useLabel = str(int(currentValue * 1e12)) + ' pA'
    xVals = timeVector
    yValsExc = np.nanmean(PSTHExcOutNan, 1)[currentValueInd, :]
    yValsInh = np.nanmean(PSTHInhOutNan, 1)[currentValueInd, :]
#     xVals = timeVector[movAvgAdj:-movAvgAdj]
#     yValsExc = moving_average(R.PSTHExc.mean(1)[currentValueInd, :], movAvgWidth)
#     yValsInh = moving_average(R.PSTHInh.mean(1)[currentValueInd, :], movAvgWidth)
    ax[0].plot(xVals, yValsExc, label=useLabel)
    ax[1].plot(xVals, yValsInh, label=useLabel)

    ax[0].set(ylabel='E FR (Hz)', ylim=(0, 9))
    ax[0].legend(loc='lower left')
    ax[1].set(xlabel='Time (s)', ylabel='I FR (Hz)', ylim=(0, 24))
# ax[0].vlines(3, *ax[0].get_ylim(), color='k', ls='--', lw=1)
# ax[1].vlines(3, *ax[1].get_ylim(), color='k', ls='--', lw=1)

WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: Mea

In [212]:
anax = ax[0]

In [50]:
anax.get_ylim()

(0.0, 7.0)

In [ ]:
anax.legend()

In [22]:
# get the QTCore PyRect object
geom = mngr.window.geometry()

x,y,dx,dy = geom.getRect()

print(x,y,dx,dy)

1997 83 640 532


In [ ]:
mngr = plt.get_current_fig_manager()
# to put it into the upper left corner for example:
# mngr.window.setGeometry(50,100,640, 545)
targetFile = ''